In [ ]:
!pip install datasets
!pip install keybert
!pip install textacy
!python -m spacy download en

In [ ]:
import spacy
import textacy

# Spacy model
nlp = textacy.load_spacy_lang("en_core_web_sm")

In [ ]:
def lemmatize(text):
    doc = nlp(text.lower())
    tokens = " ".join(token.lemma_ for token in doc)
    return tokens

def preprocess_keyphrase(text):
    tokens = []
    for t in text:
        doc = nlp(t.lower())
        doc = " ".join(token.lemma_ for token in doc)
        tokens.append(doc)
    return tokens

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments, Seq2SeqTrainer
import os
from nltk.stem import WordNetLemmatizer
import torch
from sklearn.metrics import jaccard_score
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

# Definisci il percorso di salvataggio su Drive
drive_base_path = "/content/drive/MyDrive/T5_Keyphrase_Extraction"
os.makedirs(drive_base_path, exist_ok=True)
model_save_path = os.path.join(drive_base_path, "trained_model")

# Percorsi per dataset tokenizzato e modello
tokenized_dataset_path = os.path.join(drive_base_path, "tokenized_dataset")

# Carica il dataset Inspec
# Assumiamo che ogni esempio contenga le chiavi "abstract" e "keyphrases" (quest'ultima è una lista di stringhe)
dataset = load_dataset("taln-ls2n/inspec", split="train", trust_remote_code=True)
val_dataset = load_dataset("taln-ls2n/inspec", split="validation", trust_remote_code=True)
test_dataset = load_dataset("taln-ls2n/inspec", split="test", trust_remote_code=True)

dataset = dataset.map(lambda x: {"abstract": lemmatize(x["abstract"])})
dataset = dataset.map(lambda x: {"title": lemmatize(x["title"])})
dataset = dataset.map(lambda x: {"keyphrases": preprocess_keyphrase(x["keyphrases"])})

# Preprocessamento: creiamo l'input concatenando il prompt con l'abstract
# e il target come stringa con le keyphrase separate da virgola.
def preprocess(example):
    input_text = "Extract keyphrases: " + example["title"]+". "+example["abstract"]
    # Se keyphrases è una lista, uniscile in una singola stringa separate da virgole
    if isinstance(example["keyphrases"], list):
        target_text = ", ".join(example["keyphrases"])
    else:
        target_text = example["keyphrases"]
    return {"input_text": input_text, "target_text": target_text}

# Applichiamo il preprocessamento
dataset = dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)

test_dataset = test_dataset.map(lambda x: {"abstract": lemmatize(x["abstract"])})
test_dataset = test_dataset.map(lambda x: {"title": lemmatize(x["title"])})
test_dataset = test_dataset.map(lambda x: {"keyphrases": preprocess_keyphrase(x["keyphrases"])})
test_dataset = test_dataset.map(preprocess)

In [ ]:
# Scegliamo il modello T5 (in questo esempio T5-small, ma puoi scegliere una versione più grande se necessario)
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

# Funzione per tokenizzare gli esempi
def tokenize_function(example):
    # Tokenizza l'input
    model_inputs = tokenizer(example["input_text"], max_length=512, truncation=True)
    # Tokenizza il target con il tokenizer dedicato per le sequenze di destinazione
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["target_text"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Applichiamo la tokenizzazione al dataset (processo in batch per velocità)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
val_tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Impostiamo il data collator per il sequence-to-sequence task
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# Salva il tokenized_datasets in una directory specifica
dataset.save_to_disk(os.path.join(drive_base_path, "train_tokenized_dataset"))
val_dataset.save_to_disk(os.path.join(drive_base_path, "val_tokenized_dataset"))
test_dataset.save_to_disk(os.path.join(drive_base_path, "test_tokenized_dataset"))

In [ ]:
from datasets import load_from_disk
dataset = load_from_disk(os.path.join(drive_base_path, "train_tokenized_dataset"))
val_dataset = load_from_disk(os.path.join(drive_base_path, "val_tokenized_dataset"))
test_dataset = load_from_disk(os.path.join(drive_base_path, "test_tokenized_dataset"))

In [ ]:
# Definiamo i parametri dell'addestramento
training_args = TrainingArguments(
    output_dir=os.path.join(model_save_path, "t5-inspec"),
    logging_strategy="epoch",            # Logga alla fine di ogni epoca
    per_device_train_batch_size=8,     # Batch size per l'addestramento
    num_train_epochs=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=1e-3,
    weight_decay=0.01,
)

# Utilizziamo il Seq2SeqTrainer per compiti di generazione
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Avvia l'addestramento: ad ogni epoca il trainer stamperà i log, includendo loss e le metriche (es. accuratezza)
trainer.train()

# Salva il modello fine-tuned
model_save_path = os.path.join(training_args.output_dir, "final_model")
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

In [ ]:
# Salva il modello addestrato
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

In [ ]:
# Carica il modello salvato
model = T5ForConditionalGeneration.from_pretrained(os.path.join(model_save_path, "/t5-inspec/runs/Apr11_15-20-01_1d01f3dc3706"))
tokenizer = T5Tokenizer.from_pretrained(model_save_path)

In [ ]:
# Funzione per calcolare la similarità di Jaccard
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0.0

In [ ]:
def test_model(model, tokenizer, test_dataset):
    model.eval().to("cpu")  # Imposta il modello in modalità di valutazione su CPU
    jaccard_scores = []  # Lista per memorizzare i punteggi di similarità

    for i, example in enumerate(test_dataset):
        # Tokenizza l'input (abstract)
        inputs = tokenizer(example["input_text"], return_tensors="pt", padding=True, truncation=True, max_length=512)
        # Predici le keyphrase
        with torch.no_grad():
            outputs = model.generate(**inputs, num_beams=4, max_length=128, early_stopping=True)

        # Decodifica l'output
        predicted_keyphrases = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predicted_keyphrases_list = predicted_keyphrases.split(',')  # Lista di keyphrase predette
        predicted_keyphrases_list = [phrase.strip() for phrase in predicted_keyphrases_list]  # Rimuovi spazi bianchi
        real_keyphrases_list = example["keyphrases"]

        # Calcola la similarità di Jaccard tra le keyphrase reali e quelle predette
        jaccard_score_value = jaccard_similarity(real_keyphrases_list, predicted_keyphrases_list)
        jaccard_scores.append(jaccard_score_value)

        print("Input (Abstract):", example["input_text"])
        print("Real Keyphrases:", example["keyphrases"])
        print("Predicted Keyphrases:", predicted_keyphrases_list)
        print("Jaccard Similarity:", jaccard_score_value)
        print("-" * 80)

    # Calcola la media della similarità di Jaccard
    average_jaccard_score = np.mean(jaccard_scores)
    print("Average Jaccard Similarity:", average_jaccard_score)

# Esegui il testing sul dataset di test
test_model(model, tokenizer, tokenized_test_dataset)


In [ ]:
#TEXTACY

textacy_keyphrases = []
real_keyphrases_list = []
jaccard_scores = []
for elem in test_dataset:
  doc = textacy.make_spacy_doc(elem['input_text'], lang=nlp)
  textacy_keyphrases_current = [kps for kps, weights in textacy.extract.keyterms.textrank(doc, normalize="lemma", topn=8)]
  textacy_keyphrases.append(textacy_keyphrases_current)
  real_keyphrases_list = elem["keyphrases"]
  jaccard_score_value = jaccard_similarity(real_keyphrases_list, textacy_keyphrases_current)
  jaccard_scores.append(jaccard_score_value)
  print("Real Keyphrases:", elem["keyphrases"])
  print("Predicted Keyphrases:", textacy_keyphrases[-1])
  print("Jaccard Similarity:", jaccard_score_value)

# Calcola la media della similarità di Jaccard
average_jaccard_score = np.mean(jaccard_scores)
print("Average Jaccard Similarity:", average_jaccard_score)

In [ ]:
#KEYBERT

from keybert import KeyBERT

# Istanzia il modello (puoi usare anche modelli diversi)
kw_model = KeyBERT(model='all-MiniLM-L6-v2')  # modello leggero, ma efficace

keybert_keyphrases = []
real_keyphrases_list = []
jaccard_scores = []
for elem in test_dataset:
  keybert_keyphrases_current = [kw for kw, score in kw_model.extract_keywords(elem['input_text'], keyphrase_ngram_range=(1, 5), stop_words='english', top_n=8)]
  keybert_keyphrases.append(keybert_keyphrases_current)
  real_keyphrases_list = elem["keyphrases"]
  jaccard_score_value = jaccard_similarity(real_keyphrases_list, keybert_keyphrases_current)
  jaccard_scores.append(jaccard_score_value)
  print("Real Keyphrases:", elem["keyphrases"])
  print("Predicted Keyphrases:", keybert_keyphrases[-1])
  print("Jaccard Similarity:", jaccard_score_value)

# Calcola la media della similarità di Jaccard
average_jaccard_score = np.mean(jaccard_scores)
print("Average Jaccard Similarity:", average_jaccard_score)